# Change Ringing

Change ringing is an algorithmic procedure for ringing church bells. It was invented in the 17th century by those clever British, who also gave these algorithms great names like Plain Bob Minimus, Grandsire Doubles, Reverse Canterbury Pleasure Place Doubles, etc. The algorithms all involve rotating different pairs of bells in the peal, but "...the composer's job is to be sure that (s)he has selected as far as possible the most musical sequences from the many thousands available." I was first made aware of this compositional technique by the British composer Nicky Hind.

*Note*: This notebook assumes you are familiar with the `Rotation` pattern, its 'rules' and its support function `all()`.  To learn about Rotation see the notebook [patterns.ipynb](patterns.ipynb) located in the same directory as this file.

Here are a few worthwhile videos about change ringing:

- [Mathematical Impressions: Change Ringing](https://www.youtube.com/watch?v=3lyDCUKsWZs):<br>A quick intro to change ringing and connection to math (6min).
- [Change Ringing: The Beautiful Intersection Between Math and Music](https://www.youtube.com/watch?v=f5GmUxl2NaU):<br>In depth video about connections with group theory (~29min). 
- [The Craft of Bellringing](https://www.youtube.com/watch?v=yLMiK-TMyPI):<br>Long video includes history, information about bell mechanics, ringing techniques, training, etc. (~50min). 

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
import sys
from random import randint
from musx import Rotation, Range, Score, Note, Seq, MidiFile, MidiEvent, version,\
setmidiplayer, playfile, keynum, rescale, rhythm
from musx.midi.gm import TubularBells, Flute, OrchestralHarp
print(f"musx version: {version}")

Generate midi files and automatically play them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal based midi player to use with musx.  If you dont have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")
print('OK!')

Utilities for displaying and performing change ringing examples:

In [ ]:
# a dictionary that maps bell names a,b,c... to key numbers 65,66,67...
peal = dict(zip(['a','b','c','d','e','f','g'], keynum('f5 e d c b4 a g f')))

default_rate = .2

def printdata(data):
    genlen = len(data[0])
    total = len(data) * genlen
    print(f"num bells: {genlen}, generations: {len(data)}, bell strikes: {total}")
    print(f"\nperformance time at strike rate {default_rate} is {round(default_rate*total/60,1)} minutes.")
    print(f"\n{data}")

def playbells(data, rate=default_rate, dur=.6, amp=.5, ins=TubularBells):
    meta = MidiFile.metatrack(ins={0: ins})
    if (isinstance(data[0], list)): # flatten if sublist generations
        data = [i for g in data for i in g]
    score = Score(out=Seq())
    def play(score):
        for d in data:
            k = peal[d]
            m = Note(time=score.now, pitch=k, duration=dur, amplitude=amp)
            score.add(m)
            yield rate
    score.compose(play(score))
    file = MidiFile("changeringing.mid", [meta, score.out]).write()
    print(f"Wrote '{file.pathname}'.")
    playfile(file.pathname)

print(f'playbells: {playbells}')

## Plain Hunt

In change ringing rules rotational changes almost always use just the first two rule numbers, i.e. the start index and the stepping increment of the rotation. Change ringing rotates (almost always) by pairs, so the step increment between rotations is generally 2. The start index is (almost always) the mod 2 cycle.

Even numbered bell hunting can be implemeted as a Rotation with two swapping rules: [[0, 2, 1] [1, 2, 1]]. The pattern that results from these rules is called the Plain Hunt. Plain Hunt causes a set of N elements to repeat after 2N changes, or N times through the cycle. 

As an example, here is a depiction of Plain Hunt Minumus on 4 'bells': [A, B, C, D], where 'x' marks the rotations that produces the next generation from the previous one. The first swap rule is [0, 2, 1] and it exchanges pairs  at index 0 1 then steps 2 over to swap the next pair at indexes 2 3. The second rule [1, 2, 1] does the same but starts at position 1 so alternate pairs are swapped.  For n elements, this process brings a pattern back to its original form after 2*n changes.

```
0 1 2 3
-------
A B C D    : generation 1
 x   x
B A D C    : generation 2
   x
B D A C    : generation 3
 x   x
D B C A    : generation 4
   x
D C B A    : generation 5
 x   x
C D A B    : generation 6
   x
C A D B    : generation 7
 x   x
A C B D    : generation 8
   x
A B C D    : generation 1 ...
```

The Plain Hunt is the most basic pattern for an even number of bells. This example perform the pattern on 6 bells and ends with the repetition of the first generation of notes. The generations will appear as sublists in the printout:

In [ ]:
def plain_hunt():
    return [[0, 2, 1], [1, 2, 1]]

rules = plain_hunt()
print('rules:', rules)

bells = Rotation(['a','b','c','d','e','f'], rules)
data = bells.all(grouped=True, wrapped=True)
printdata(data)
playbells(data)

## Plain Bob

Plain Bob builds on the Plain Hunt: n-1 repetions of cycle(0,1) followed by a "dodge" on the nth: cycle(0,2), which causes the rotation to start at the 2nd index instead of the first, this stops the return of the pattern, which finally repeats after 2n*(n-1) changes:

In [ ]:
def dodge(start, step):
    """returns a 'dodged' cycle, i.e. instead of 0,1,1 its 0,x,1."""
    return [[0, 2, 1], [start, step, 1]]

def plain_bob(n):
    rules = []
    for _ in range(n - 1):
        rules.extend(plain_hunt())
    return rules + dodge(2, 2)

rules = plain_bob(4)
print('rules:', rules)

bells = Rotation(['a','b','c','d','e','f'], plain_bob(4))
data = bells.all(grouped=True, wrapped=True)
printdata(data)
playbells(data)

## Call Bob
Call Bob builds on Plain Bob. It's n-2 repetitions of Plain Bob followed by a plain bob whose dodge is different: cycle(1,3). The total number of changes become 3*(2n*(n-1)). So for 6 bells (Call Bob Minumus), the pattern repeats after 3*60 or 180 changes.

In [ ]:
def call_bob(n):
    rules = []
    for _ in range(n - 2):
        rules.extend(plain_bob(n))
    for _ in range(n - 1):
        rules.extend(plain_hunt())
    rules.extend(dodge(1, 3))
    return rules

rules = call_bob(6)
print('rules:', rules)

bells = Rotation(['a','b','c','d','e','f'], call_bob(6))
data = bells.all(grouped=True, wrapped=True)
printdata(data)
playbells(data)

## Call Single

Call Single builds on Call Bob, but the very last dodge of 1,3 is replaced by a rotation of just the last two elements, which causes the process to double (360 changes for 6 bells). 

<b>Note</b>: performing this example at the default rate (.2 sec) will take over 7 minutes =;) If you listen for sub-patterns and slow changes it can be a lovely experience. You will know you are halfway thru when you hear a generation of completely ascending notes (i.e. the inversion of the first generation).

In [ ]:
def call_single(n):
    rules = []
    for _ in range(2):
        rules.extend(call_bob(n))
    for _ in range(n - 2):
        rules.extend(plain_bob(n))
    for _ in range(n - 1):
        rules.extend(plain_hunt())
    rules.extend(dodge(n - 2, 2))
    return rules

rules = call_single(6)
print('rules:', rules)

bells = Rotation(['a','b','c','d','e','f'], call_single(6))
data = bells.all(grouped=True, wrapped=True)
printdata(data)
playbells(data)

## Grandsire 

Grandsire rotates an <u>odd</u> number of bells based on a simple deviation to the plain hunt. *Grandsire doubles* is rung on five bells, *grandsire triples* on seven, *grandsire caters* on nine and *grandsire cinques* on eleven:

In [ ]:
def grandsire(n):
    rules = []
    rules.append([0, 3, 1])
    for i in range(n - 1):
        rules.extend([[1, 2, 1], [0, 2, 1]])
    rules.append([1, 2, 1])
    return rules

rules = grandsire(7)
print('rules:', rules)

bells = Rotation(['a','b','c','d','e','f','g'], grandsire(7))
data = bells.all(grouped=True)
printdata(data)
playbells(data)

## Roll your own Rotations

The Rotation in this example uses its own invented rules. In addition, the first element in the Rotation isn't a single value (e.g. a bell), it's a Range subpattern that descends by whole step. The subpattern has a period of 1 so it's descent will be 'spread out' over various points in the texture. Finally, the `descend()` composer treats the entire Rotation as a two voice composition by assigning the descent and accompanyment with different instruments and characteristics:

In [ ]:
melody = Rotation([ Range(keynum("c6"), 0, -2, period=1), 
                   'g2', 'cs4', 'b4', 'f3'], [[0, 1, 1,], [0, 1, 2]])

def descend(score, reps, mel, rate):
    amp2vel = lambda amp: int(rescale(amp, 0, 1, 0, 127))
    prev = 0
    for _ in range(reps):
        x = mel.next()
        # whole-tone descent (legato flute)
        if isinstance(x, int):
            k, a, d, c = x, amp2vel(.7), rate*4, 0
            # flute creates legato effect using on and off messages so the
            # next note starts immediately after the previous note stops.
            score.add(MidiEvent.note_off(channel=c, keynum=prev, velocity=127, time=score.now))
            score.add(MidiEvent.note_on(channel=c, keynum=k, velocity=a, time=score.now))
            prev = k
        # accompaniment (plucked harp)            
        else:                  
            k, a, d, c = keynum(x), .36, rate*2.5, 1
            score.add( Note(time=score.now, pitch=k, duration=d, amplitude=a, instrument=c))
        yield rate
    # all done, turn last note off.
    score.add(MidiEvent.note_off(channel=0, keynum=prev, velocity=127, time=score.now+rate))
        
meta = MidiFile.metatrack(ins={0: Flute, 1: OrchestralHarp})
score = Score(out=Seq())
score.compose( descend(score, 69, melody, .5))
file = MidiFile("changeringing.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname) 

This example generates chords from change ringing rules:

In [ ]:
pbc = Rotation([0, 3, 4, 7, 8, 11], plain_bob(6))

def plainbobchords(score, reps, pat, rate, dur, knum, amp, step):
    for _ in range(reps):
        k = knum
        for i in pat.next(6):
            if i != 0:
                note = Note(time=score.now, duration=dur, pitch=k, amplitude=amp)
                score.add(note)
                k += i
        knum += step
        yield rate
        
score = Score(out=Seq())
score.compose(plainbobchords(score, 60, pbc, .6, .6, 48, .7, 0))
file = MidiFile("changeringing.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

The same but with two layers:

In [ ]:
# Two layers of plain bob chords

x = Rotation([0, 3, 4, 7, 8, 11], plain_bob(6))
y = Rotation([0, 3, 4, 7, 8, 11], plain_bob(6))
score = Score(out=Seq())
score.compose([plainbobchords(score, 60, x, .7, 2, 80, .3, -1),
               plainbobchords(score, 60, y, .7, 2, 20, .3,  1)])
file = MidiFile("changeringing.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)